In [2]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
zhipuai.api_key = os.environ['ZHIPUAI_API_KEY']

# 加载 PDF
loaders_chinese = [
    PyMuPDFLoader("/workspaces/LLM_Project/database/knowledge_db/pumpkin_book.pdf") # 南瓜书
    # 大家可以自行加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)
!pip install sentence-transformers
embedding = HuggingFaceEmbeddings(model_name="bert-base-chinese")

persist_directory = '../../data_base/vector_db/chroma'

!rm -rf '../../data_base/vector_db/chroma'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.0 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: pip install --upgrade pip


/opt/conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name bert-base-chinese. Creating a new one with MEAN pooling.


In [3]:
vectordb = Chroma.from_documents(
    documents=split_docs[:200], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

In [4]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：200


In [5]:
question="什么是机器学习"

sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [6]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
先解释“数据决定模型效果的上限”，其中数据是指从数据量和特征工程两个角度考虑。从数据量的
角度来说，通常数据量越大模型效果越好，因为数据量大即表示累计的经验多，因此模型学习到的经验也
多，自然表现效果越好。例如以上举例中如果训练集中含有相同颜色但根蒂不蜷缩的坏瓜，模型 a 学到真
相的概率则也会增大；从特征工程的角度来说，通常对特征数值化越合理，特征收集越全越细致，模型效
果通常越好，因为此时模型
--------------
检索到的第1个内容: 
值范围通常是整个实数域 R，即 Y = R。
无论是分类还是回归，机器学习算法最终学得的模型都可以抽象地看作为以样本 x 为自变量，标记 y
为因变量的函数 y = f(x)，即一个从输入空间 X 到输出空间 Y 的映射。例如在学习西瓜的好坏时，机器
学习算法学得的模型可看作为一个函数 f(x)，给定任意一个西瓜样本 xi = (青绿; 蜷缩; 清脆)，将其输入
进函数即可计算得到一个输出 yi 
--------------
检索到的第2个内容: 
模型：机器学习的一般流程如下：首先收集若干样本（假设此时有 100 个），然后将其分为训练样本
（80 个）和测试样本（20 个），其中 80 个训练样本构成的集合称为“训练集”，20 个测试样本构成的集合
称为“测试集”，接着选用某个机器学习算法，让其在训练集上进行“学习”（或称为“训练”），然后产出
得到“模型”（或称为“学习器”），最后用测试集来测试模型的效果。执行以上流程时，表示我们已经默
--------------


In [6]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [7]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我
--------------
MMR 检索到的第1个内容: 
→_→
欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解 第 2 版》
←_←
12.7.4 式 (12.60) 的推导 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
149
12.7.5 经验损失最小化 . . . . . . . . . . . . . . . . . . . . . . 
--------------
MMR 检索到的第2个内容: 
46
5.6.2
深度学习的起源 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
46
5.6.3
怎么理解特征学习
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
46
第 6 章 支持向量机
47
--------------


In [9]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModel
from zhipuai_llm import ZhipuAILLM
llm = ZhipuAILLM(model="chatglm_std", temperature=0)
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
# 声明一个检索式问答链


persist_directory = '../../data_base/vector_db/chroma'
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = " 什么是磨菇书？"

In [10]:
result = qa_chain({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

LLM 对问题的回答：" 南瓜书是一个在线的书籍共享平台，用户可以在上面免费阅读和下载各种类型的电子书。这个平台提供了丰富的书籍资源，包括小说、教材、参考书等，支持多种文件格式，方便用户阅读。感谢您的提问！"


In [8]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

IndexError: list index out of range